## 场内基金套利策略
### 简介：通过对场内LOF基金进行监测，发现其是否与场外净值有较大偏差来进行套利
### 操作方式：
1.查看场外LOF基金前一日净值与当日实时估算净值  
2.查看场内该基金的实时交易价格  
3.判断实时交易价格是否偏离实时估算净值大于100bps  
4.判断交易量是否较前几日有明显放大  
5.在首先满足步骤3的条件下，发送信号。

### 一期计划：
实现信号功能：能够较为准确的发现套利机会，程序能够在交易时间段内持续监控  
可改进项：通过机器学习多分类来解决识别误差问题

### 二期计划：
实现信息发送功能：发送至微信或者邮件中

### 三期计划：
实现自动化交易功能。



In [1]:
###1.连接各类数据接口

In [10]:
import math
import requests,json,time
import easyquotation as eq
import numpy as np
import pandas as pd
import sqlalchemy as sql

In [11]:
#显示所有列
pd.set_option('display.max_columns', None)

#显示所有行
pd.set_option('display.max_rows', None)

In [12]:
class fundFeatures():
    def __init__(self, code, amplitude,turnover,*args):
        self.code = code
        self.amplitude = amplitude
        self.turnover = turnover
        self.esval,self.gain,self.pre_price = self.get_estimated_netval()
        self.amplitude,self.turnover,self.now,self.bid1,self.bid1_volume,self.bid2,self.bid2_volume,self.ask1,self.ask1_volume,self.ask2,self.ask2_volume=self.get_now_data()
        
    def change(self):
        self.esval,self.gain,self.pre_price = self.get_estimated_netval()
        self.amplitude,self.turnover,self.now,self.bid1,self.bid1_volume,self.bid2,self.bid2_volume,self.ask1,self.ask1_volume,self.ask2,self.ask2_volume=self.get_now_data()
        
    def get_estimated_netval(self):
        t = time.time()
        timestamp = round(t * 1000)
        Headers = {'content-type':'application/json','User-Agent': 'Apache-HttpClient/4.5.2 (Java/1.8.0_102)'}
        TTurl = "http://fundgz.1234567.com.cn/js/" + str(self.code) +  ".js?rt=" + str(timestamp)
        r = requests.get(TTurl, headers=Headers)
        GetMsg = r.text
        if "fundcode" in GetMsg:
            estimated_netval = GetMsg.split(",")[4].split(":")[1].replace('"','')
            gain = GetMsg.split(",")[5].split(":")[1].replace('"','')
            pre_price=  GetMsg.split(",")[3].split(":")[1].replace('"','')
            return float(estimated_netval),float(gain),float(pre_price)
        else:
            return 0,0,0
    def get_now_data(self):
        code = str(self.code)
        quotation =eq.use('tencent')
        fund_dict =quotation.real(code)
        return fund_dict[code]['振幅'],fund_dict[code]['成交额(万)'],fund_dict[code]['now'],fund_dict[code]['bid1'],fund_dict[code]['bid1_volume'],fund_dict[code]['bid2'],fund_dict[code]['bid2_volume'],fund_dict[code]['ask1'],fund_dict[code]['ask1_volume'],fund_dict[code]['ask2'],fund_dict[code]['ask2_volume']
        
    def as_dict(self):
        return {'code': self.code, 'amplitude': self.amplitude, 'turnover': self.turnover,'gain': self.gain,'pre_price': self.pre_price,
               'es_value': self.esval,'now': self.now,'bid1': self.bid1,'bid1_volume': self.bid1_volume,'bid2': self.bid2,'bid2_volume': self.bid2_volume,'ask1': self.ask1,'ask1_volume': self.ask1_volume,'ask2': self.ask2,'ask2_volume':self.ask2_volume}

In [13]:
def initFunds():
    #读取fof基金列表
    df=pd.read_csv('/Users/zhangming/Work/invest/etf.csv')
    tmp_funds_list = df['main_code'].to_list()
    str_funds_list= [str(x) for x in tmp_funds_list]
    #对基金进行选择操作，逻辑成交额大于10万，并且振幅大于3个点
    quotation = eq.use('tencent') # 新浪 ['sina'] 腾讯 ['tencent', 'qq']
    funds_dict = quotation.stocks(str_funds_list)
    valid_funds = []
    #初始化基金数据，获取实时涨跌，振幅，并转为pd格式
    for key,value in funds_dict.items():
        obj = fundFeatures(key,value['振幅'],value['成交额(万)'])
        valid_funds.append(obj)
    return valid_funds
#valid_funds = sorted(valid_funds, key=lambda valid_fund: valid_fund.amplitude,reverse=True)

In [14]:
#更新基金实时变动
def get_now_funds(valid_funds):
    for item in valid_funds:
        item.change()
    real_fund = pd.DataFrame([x.as_dict() for x in valid_funds])
    real_fund = real_fund.drop(real_fund[real_fund.es_value == 0 ].index)
    real_fund=real_fund[(real_fund['turnover'] > 0 )|(real_fund['bid1']>0)]
    real_fund['change_pct'] = (real_fund['now']-real_fund['es_value'])/real_fund['es_value'] *100
    return real_fund

    

In [15]:
valid_funds = initFunds()

#167508,161912,160526

AttributeError: module 'requests' has no attribute 'session'

In [9]:
#获得实时基金变动
res = get_now_funds(valid_funds)
res[(res['turnover']>=50000)&(res['turnover']<=5000000)].sort_values(by=['amplitude','change_pct'],ascending=[False,False])

NameError: name 'valid_funds' is not defined

In [35]:
now_date = time.strftime('%Y-%m-%d',time.localtime())
res['date']=now_date
file_path='/Users/zhangming/Work/invest/'+now_date+'.csv'
res.to_csv(file_path)

In [247]:
if __name__ == '__main__':
    #初始化基金
    valid_funds = initFunds()
    
    #获得实时基金变动
    res = get_now_funds(valid_funds)
    res


In [1]:
res = get_now_funds(valid_funds)
res

NameError: name 'get_now_funds' is not defined

In [ ]:
for item in valid_funds:
    item.change()
real_fund = pd.DataFrame([x.as_dict() for x in valid_funds])
real_fund = real_fund.sort_values(by=['amplitude','turnover'],ascending=[False,False])
real_fund = real_fund.drop(real_fund[real_fund.es_value == 0 ].index)
real_fund=real_fund[(real_fund['turnover'] > 0 )|(real_fund['bid1']>0)]
real_fund.reset_index(drop=True, inplace=True)
real_fund['change_pct'] = (real_fund['now']-real_fund['es_value'])/real_fund['es_value'] *100



In [54]:
quotation = eq.use('tencent') # 新浪 ['sina'] 腾讯 ['tencent', 'qq']
quotation.stocks(['161030', '160516'])
#选相应波动基金
for 
#tmp_dict = quotation.market_snapshot(prefix=True) # prefix 参数指定返回的行情字典中的股票代码 key 是否带 sz/sh 前缀

{'161030': {'name': '体育',
  'code': '161030',
  'now': 1.038,
  'close': 1.045,
  'open': 0.941,
  'volume': 933200.0,
  'bid_volume': 330100,
  'ask_volume': 603100.0,
  'bid1': 1.038,
  'bid1_volume': 38000,
  'bid2': 1.035,
  'bid2_volume': 5000,
  'bid3': 1.014,
  'bid3_volume': 10000,
  'bid4': 0.964,
  'bid4_volume': 3700,
  'bid5': 0.963,
  'bid5_volume': 11000,
  'ask1': 1.04,
  'ask1_volume': 10100,
  'ask2': 1.042,
  'ask2_volume': 105200,
  'ask3': 1.045,
  'ask3_volume': 18600,
  'ask4': 1.08,
  'ask4_volume': 10100,
  'ask5': 1.09,
  'ask5_volume': 1000,
  '最近逐笔成交': '',
  'datetime': datetime.datetime(2021, 3, 5, 16, 15, 3),
  '涨跌': -0.007,
  '涨跌(%)': -0.67,
  'high': 1.042,
  'low': 0.941,
  '价格/成交量(手)/成交额': '1.038/9332/897730',
  '成交量(手)': 933200,
  '成交额(万)': 900000.0,
  'turnover': None,
  'PE': None,
  'unknown': '',
  'high_2': 1.042,
  'low_2': 0.941,
  '振幅': 9.67,
  '流通市值': None,
  '总市值': None,
  'PB': 0.0,
  '涨停价': 1.15,
  '跌停价': 0.941,
  '量比': 1.93,
  '委差': -773.0

In [ ]:

# def GetFundData(fundId,GetTimestamp):
#     Headers = {'content-type':'application/json','User-Agent': 'Apache-HttpClient/4.5.2 (Java/1.8.0_102)'}
#     TTurl = "http://fundgz.1234567.com.cn/js/" + str(fundId) +  ".js?rt=" + str(GetTimestamp)
#     r = requests.get(TTurl, headers=Headers)
#     GetMsg = r.text
#     print(GetMsg)
#     if "fundcode" in GetMsg:
#         FundID = GetMsg.split(",")[0].split(":")[1]
#         FundName = GetMsg.split(",")[1].split(":")[1]
        
#         Gain = GetMsg.split(",")[5].split(":")[1]
#         # print(GetMsg.split(",")[6].split("}")[0])
#         currentTime = GetMsg.split(",")[6].split("}")[0].split("\"gztime\":")[1]
#         # GetData = "基金名称:"+ FundName + "ID" + FundID + "涨跌幅:" + Gain + "当前时间:" + currentTime
#         # GetData =  FundName + " " + FundID + " " + Gain
#         # GetData = "基金名称:"+ FundName + "涨跌幅:" + Gain + "当前时间:" + currentTime
#         GetData = "基金名称:"+ FundName + " "+ "涨跌幅:" + Gain + " "+ "当前时间:" + currentTime
#     return GetData
# #获取时间戳
# def GetTime():
#     t = time.time()
#     Timestamp = round(t * 1000)
#     # print(Timestamp)
#     return Timestamp
# GetTimestamp = GetTime()
# GetFundData('161030',GetTimestamp)